In [16]:

import c302
from c302.NeuroMLUtilities import analyse_connections

all_data = {}
all_data["Values"] =["Num Cells", "Num muscles", "Num conns", "#N->Musc"]

readers = {"SpreadsheetData": "c302.SpreadsheetDataReader", 
           "UpdSpreadsheetData": "c302.UpdatedSpreadsheetDataReader",
           "UpdSpreadsheetData2": "c302.UpdatedSpreadsheetDataReader2",
           "OpenWorm": "c302.OpenWormReader",
           "WormNeuroAtlas": "c302.WormNeuroAtlasReader"}

for name, reader in readers.items():

    print("\n****** Importing dataset %s using %s ******"% (name, reader))
    exec("from %s import read_data, read_muscle_data"%reader)

    cells, neuron_conns = read_data(include_nonconnected_cells=True)
    neurons2muscles, muscles, muscle_conns = read_muscle_data()

    all_data[name] =[len(cells),len(muscles), len(neuron_conns), len(neurons2muscles)]




****** Importing dataset SpreadsheetData using c302.SpreadsheetDataReader ******
c302      >>>  Opened Excel file..: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/CElegansNeuronTables.xls
c302      >>>  Opened Excel file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/CElegansNeuronTables.xls

****** Importing dataset UpdSpreadsheetData using c302.UpdatedSpreadsheetDataReader ******
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edgelist.csv
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edgelist.csv

****** Importing dataset UpdSpreadsheetData2 using c302.UpdatedSpreadsheetDataReader2 ******
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-packages/c302/data/herm_full_edgelist_MODIFIED.csv
c302      >>>  Opened file: /opt/homebrew/anaconda3/envs/py39n/lib/python3.9/site-pac

In [17]:
import pandas as pd
import numpy as np

df_all = pd.DataFrame(all_data)
df_all.set_index("Values")


,SpreadsheetData,UpdSpreadsheetData,UpdSpreadsheetData2,OpenWorm,WormNeuroAtlas
Values,,,,,
Num Cells,302,302,302,302,300
Num muscles,97,95,95,95,0
Num conns,3363,5806,5821,5805,1650
#N->Musc,127,254,254,159,0
